In [15]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift
from sklearn import preprocessing

In [16]:
df = pd.read_excel('./data sets/titanic.xls')
orginal_df =  pd.DataFrame.copy(df)
df.head(5)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [17]:
df.drop(['body', 'name'], 1, inplace=True)

C:\Users\nwukw\AppData\Local\Temp\ipykernel_13276\3798999643.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['body', 'name'], 1, inplace=True)


In [18]:
df.fillna(0, inplace=True)

In [19]:
def handle_non_numeric_data(df):
    columns  = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]
        
        if df[column].dtype != np.int64  and df[column].dtype !=np.float64:
            column_contents =  df[column].values.tolist()
            unique_elements  = set(column_contents)
            x=0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))      

    return df

In [20]:
df =  handle_non_numeric_data(df)

In [21]:
X =  np.array(df.drop(['survived'], 1).astype(float))
X =  preprocessing.scale(X)
y = np.array(df['survived'])

C:\Users\nwukw\AppData\Local\Temp\ipykernel_13276\1188670964.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X =  np.array(df.drop(['survived'], 1).astype(float))


In [22]:
clf =MeanShift()
clf.fit(X)

MeanShift()

In [23]:
labels = clf.labels_
cluster_centers = clf.cluster_centers_
orginal_df['cluster_group'] =  np.nan

In [24]:
for i in range(len(X)):
    orginal_df['cluster_group'].iloc[i] = labels[i]

n_clusters_ =  len(np.unique(labels))
survival_rates = {}

for i in range(n_clusters_):
    temp_df =  orginal_df[orginal_df['cluster_group'] == float(i) ]
    survival_cluster = temp_df[(temp_df['survived'])==1]
    survival_rate = len(survival_cluster)/len(temp_df)
    survival_rates[i] =  survival_rate

print(survival_rates)

{0: 0.38589540412044376, 1: 0.0, 2: 0.9090909090909091, 3: 0.1, 4: 0.6666666666666666}


C:\Users\nwukw\AppData\Local\Temp\ipykernel_13276\2923417100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orginal_df['cluster_group'].iloc[i] = labels[i]


In [31]:
print(orginal_df[(orginal_df['cluster_group']==4)])

     pclass  survived                         name   sex   age  sibsp  parch  \
183       1         1       Lesurer, Mr. Gustave J  male  35.0      0      0   
249       1         1  Ryerson, Master. John Borie  male  13.0      2      2   
252       1         0   Ryerson, Mr. Arthur Larned  male  61.0      1      3   

       ticket      fare            cabin embarked boat  body  \
183  PC 17755  512.3292             B101        C    3   NaN   
249  PC 17608  262.3750  B57 B59 B63 B66        C    4   NaN   
252  PC 17608  262.3750  B57 B59 B63 B66        C  NaN   NaN   

                           home.dest  cluster_group  
183                              NaN            4.0  
249  Haverford, PA / Cooperstown, NY            4.0  
252  Haverford, PA / Cooperstown, NY            4.0  
